## Conexão com google colab
---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Importes necessários
---

In [ ]:
%pip install mlflow
%pip install boto3 psycopg2
%pip install ultralytics

import ultralytics
ultralytics.checks()

Ultralytics 8.3.61 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.7/107.7 GB disk)


## Treinamento de placa de carros

---



In [ ]:
import os
import shutil
import random

# Defina os diretórios de origem e destino
base_dir = '/content/drive/MyDrive/ColabNotebooks/tcc_mba/treinamento_yolo_placa/training'
dest_dir_base = '/content/drive/MyDrive/ColabNotebooks/tcc_mba/treinamento_yolo_placa/training_800'
categories = ['cars-br', 'cars-me', 'motorcycles-br', 'motorcycles-me']

# Quantidades desejadas
total_train = 560
total_val = 160
total_test = 80

# Função para organizar as amostras por categoria
def organizar_amostras():
    quantidades_desejadas = {
        'train': total_train // len(categories),
        'val': total_val // len(categories),
        'test': total_test // len(categories)
    }

    # Criar as pastas de destino
    for tipo in ['train', 'val', 'test']:
        os.makedirs(os.path.join(dest_dir_base, tipo, 'images'), exist_ok=True)
        os.makedirs(os.path.join(dest_dir_base, tipo, 'labels'), exist_ok=True)

    for category in categories:
        imagens = [f for f in os.listdir(os.path.join(base_dir, category)) if f.endswith('.jpg')]

        # Verificar se há imagens suficientes
        if len(imagens) < quantidades_desejadas['train']:
            print(f"Categoria '{category}' não possui amostras suficientes para treino. Utilizando o máximo disponível ({len(imagens)} amostras).")
            quantidades_desejadas['train'] = len(imagens)

        # Embaralhar as imagens
        random.shuffle(imagens)

        # Copiar as amostras para as pastas de treino, validação e teste
        for tipo in ['train', 'val', 'test']:
            # Determinar quantas amostras copiar
            num_samples = quantidades_desejadas[tipo]
            for i in range(num_samples):
                if i < len(imagens):
                    # Copiar imagem
                    src_image = os.path.join(base_dir, category, imagens[i])
                    dest_image = os.path.join(dest_dir_base, tipo, 'images', imagens[i])
                    shutil.copy(src_image, dest_image)

                    # Copiar label correspondente
                    src_label = os.path.join(base_dir, category, imagens[i].replace('.jpg', '.txt'))
                    dest_label = os.path.join(dest_dir_base, tipo, 'labels', imagens[i].replace('.jpg', '.txt'))
                    shutil.copy(src_label, dest_label)

        # Print para verificar o número de amostras copiadas
        print(f"Categoria '{category}': {quantidades_desejadas['train']} amostras para treino, {quantidades_desejadas['val']} amostras para validação, {quantidades_desejadas['test']} amostras para teste.")

# Testar a função
organizar_amostras()

Categoria 'cars-br': 140 amostras para treino, 40 amostras para validação, 20 amostras para teste.
Categoria 'cars-me': 140 amostras para treino, 40 amostras para validação, 20 amostras para teste.
Categoria 'motorcycles-br': 140 amostras para treino, 40 amostras para validação, 20 amostras para teste.
Categoria 'motorcycles-me': 140 amostras para treino, 40 amostras para validação, 20 amostras para teste.


In [ ]:
import mlflow
import mlflow.pytorch
from ultralytics import YOLO
import os
import time

os.environ['WANDB_DISABLED'] = 'true'

# Configurações do AWS
os.environ['AWS_ACCESS_KEY_ID'] = ""
os.environ['AWS_SECRET_ACCESS_KEY'] = ""
os.environ['AWS_DEFAULT_REGION'] = ''

EXPERIMENT_NAME = "YOLOv8 Full Parameter Tracking"
os.environ["MLFLOW_TRACKING_URI"] = ""
mlflow.set_experiment(EXPERIMENT_NAME)

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print("experiment_id:", experiment.experiment_id)
print("artifact_location:", experiment.artifact_location)
print("lifecycle_stage:", experiment.lifecycle_stage)

try:
    with mlflow.start_run(experiment_id=experiment.experiment_id, run_name="detect_place_yolo") as run:
        model = YOLO('yolov8x.pt')

        # Define as tags para o experimento
        tags = {"Model": "Yolov8", "User": "RogLopes",
                "TCC": "Detect Plate", "Approach": "best_model"}
        mlflow.set_tags(tags)

        # Loga o modelo treinado
        mlflow.pytorch.log_model(model, "model")

        # Treina o modelo YOLOv8
        results = model.train(
            data='/content/drive/MyDrive/ColabNotebooks/tcc_mba/treinamento_yolo_placa/training_800/data.yaml',  # Caminho do arquivo YAML do dataset
            epochs=100,
            batch=16,
            imgsz=640
        )

        # Valida o modelo
        val_results = model.val('/content/drive/MyDrive/ColabNotebooks/tcc_mba/treinamento_yolo_placa/training_800/data.yaml')

except mlflow.exceptions.MlflowException as e:
    print(f"Erro no MLflow: {str(e)}")
except Exception as e:
    print(f"Erro inesperado: {str(e)}")

print("Training completed and fully tracked with MLflow.")

experiment_id: 1
artifact_location: s3://mlflow-detection-plate-tracking/1
lifecycle_stage: active


2025/01/02 22:33:42 WARNING mlflow.utils.requirements_utils: Found torch version (2.5.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.5.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/01/02 22:33:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/content/drive/MyDrive/ColabNotebooks/tcc_mba/treinamento_yolo_placa/training_800/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=F

train: Scanning /content/drive/MyDrive/ColabNotebooks/tcc_mba/treinamento_yolo_placa/training_800/train/labels... 560 images, 0 backgrounds, 0 corrupt: 100%|██████████| 560/560 [00:07<00:00, 77.17it/s]


train: New cache created: /content/drive/MyDrive/ColabNotebooks/tcc_mba/treinamento_yolo_placa/training_800/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

    100/100      13.2G      0.276     0.1604     0.7879         16        640: 100%|██████████| 35/35 [00:43<00:00,  1.25s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.62it/s]

                   all        160        160          1          1      0.995      0.979



100 epochs completed in 1.775 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 136.7MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 136.7MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.34it/s]


                   all        160        160          1          1      0.995       0.98
Speed: 0.4ms preprocess, 15.5ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/train2
🏃 View run detect_place_yolo at: http://ec2-34-227-225-182.compute-1.amazonaws.com:5000/#/experiments/1/runs/c100103797e54439ab8fa4bd06e58858
🧪 View experiment at: http://ec2-34-227-225-182.compute-1.amazonaws.com:5000/#/experiments/1
MLflow: results logged to http://ec2-34-227-225-182.compute-1.amazonaws.com:5000
MLflow: disable with 'yolo settings mlflow=False'
Erro inesperado: 'str' object is not callable
Training completed and fully tracked with MLflow.
